In [1]:
#@title 🎬 Wan2GP Complete Setup - Google Colab Optimized with Python 3.10.9
import os
import sys
import platform
import subprocess
import gc
import json
import time
from IPython.display import display, HTML, clear_output

# Global variables
DETECTED_PLATFORM = None
TORCH_VERSION = None
CUDA_VERSION = None
DETECTED_VRAM = 8.0
BEST_ATTENTION = "sdpa"
optimizations = {}
PYTHON_PATH = None
PIP_PATH = None
USES_VIRTUAL_ENV = False

def detect_platform():
    """Comprehensive platform detection"""
    print("🔍 Detecting cloud platform...")
    if 'google.colab' in sys.modules:
        print("✅ Detected: Google Colab")
        return 'colab'
    print("❓ Unknown platform - Using default setup")
    return 'unknown'

def install_python310():
    """Install Python 3.10.9 on Colab"""
    print("🐍 Installing Python 3.10.9...")
    try:
        # Add deadsnakes PPA and install Python 3.10
        subprocess.run(['apt', 'update'], check=True, capture_output=True)
        subprocess.run(['apt', 'install', '-y', 'software-properties-common'], check=True, capture_output=True)
        subprocess.run(['add-apt-repository', '-y', 'ppa:deadsnakes/ppa'], check=True, capture_output=True)
        subprocess.run(['apt', 'update'], check=True, capture_output=True)
        subprocess.run(['apt', 'install', '-y', 'python3.10', 'python3.10-venv', 'python3.10-dev'], check=True, capture_output=True)

        # Verify installation
        result = subprocess.run(['python3.10', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Python 3.10 installed: {result.stdout.strip()}")
            return True
        else:
            raise Exception("Python 3.10 installation verification failed")

    except Exception as e:
        print(f"❌ Failed to install Python 3.10: {e}")
        return False

def setup_colab_venv():
    """Create isolated Python 3.10.9 environment in Colab"""
    global PYTHON_PATH, PIP_PATH, USES_VIRTUAL_ENV

    print("🔧 Setting up isolated Python 3.10.9 environment...")

    # First install Python 3.10
    if not install_python310():
        print("🔄 Falling back to system Python with --user installs...")
        PYTHON_PATH = sys.executable
        PIP_PATH = 'pip'
        USES_VIRTUAL_ENV = False
        return False

    try:
        # Create virtual environment in /content for persistence
        venv_path = '/content/wan2gp_env'

        # Remove existing venv if present
        if os.path.exists(venv_path):
            print("🔄 Removing existing virtual environment...")
            subprocess.run(['rm', '-rf', venv_path], check=True)

        print("📦 Creating Python 3.10.9 virtual environment...")
        subprocess.run(['python3.10', '-m', 'venv', venv_path], check=True)

        # Set paths to virtual environment
        PYTHON_PATH = f'{venv_path}/bin/python'
        PIP_PATH = f'{venv_path}/bin/pip'

        # Verify virtual environment
        if os.path.exists(PYTHON_PATH):
            # Upgrade pip in virtual environment
            subprocess.run([PIP_PATH, 'install', '--upgrade', 'pip'], check=True)

            # Get Python version in venv
            result = subprocess.run([PYTHON_PATH, '--version'], capture_output=True, text=True)
            print(f"✅ Virtual environment created: {result.stdout.strip()}")
            USES_VIRTUAL_ENV = True
            return True
        else:
            raise Exception("Virtual environment creation failed")

    except Exception as e:
        print(f"⚠️ Virtual environment setup failed: {e}")
        print("🔄 Falling back to system Python with --user installs...")

        # Fallback to system Python with --user installs
        PYTHON_PATH = sys.executable
        PIP_PATH = 'pip'
        USES_VIRTUAL_ENV = False
        return False

# Execute setup
try:
    DETECTED_PLATFORM = detect_platform()
    USES_VIRTUAL_ENV = setup_colab_venv()

    print(f"\n📊 Setup Summary:")
    print(f"🐍 Python executable: {PYTHON_PATH}")
    print(f"📦 Pip executable: {PIP_PATH}")
    print(f"📍 Platform: {DETECTED_PLATFORM}")
    print(f"🏠 Virtual Environment: {'Yes (Python 3.10.9)' if USES_VIRTUAL_ENV else 'No (System + --user)'}")

except Exception as e:
    print(f"❌ Setup failed: {e}")
    DETECTED_PLATFORM = 'unknown'
    USES_VIRTUAL_ENV = False
    PYTHON_PATH = sys.executable
    PIP_PATH = 'pip'


🔍 Detecting cloud platform...
✅ Detected: Google Colab
🔧 Setting up isolated Python 3.10.9 environment...
🐍 Installing Python 3.10.9...
✅ Python 3.10 installed: Python 3.10.12
🔄 Removing existing virtual environment...
📦 Creating Python 3.10.9 virtual environment...
✅ Virtual environment created: Python 3.10.12

📊 Setup Summary:
🐍 Python executable: /content/wan2gp_env/bin/python
📦 Pip executable: /content/wan2gp_env/bin/pip
📍 Platform: colab
🏠 Virtual Environment: Yes (Python 3.10.9)


In [2]:
#@title 🎮 Step 2: GPU Detection & PyTorch 2.6.0 + CUDA 12.4 Installation
import subprocess
import sys

def detect_gpu_generation():
    """Detect GPU generation with comprehensive error handling"""
    print("\n🎮 Detecting GPU generation...")
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader,nounits'],
                               capture_output=True, text=True, timeout=30)
        if result.returncode == 0:
            gpu_info = result.stdout.strip().split(',')
            gpu_name = gpu_info.strip() if gpu_info else "Unknown GPU"
            vram_mb = int(gpu_info[1].strip()) if len(gpu_info) > 1 and gpu_info[1].strip().isdigit() else 8192
            vram_gb = vram_mb / 1024
            print(f"GPU detected: {gpu_name}")
            print(f"VRAM: {vram_gb:.1f} GB")
            print("⚡ Using PyTorch 2.6.0 + CUDA 12.4 (Required for WanGP)")
            return "2.6.0", "cu124", "https://download.pytorch.org/whl/test/cu124", vram_gb
        else:
            raise Exception("nvidia-smi failed")
    except Exception as e:
        print(f"⚠️ GPU detection failed: {e}")
        print("📦 Defaulting to PyTorch 2.6.0 + CUDA 12.4")
        return "2.6.0", "cu124", "https://download.pytorch.org/whl/test/cu124", 8.0

def install_pytorch():
    """Install PyTorch 2.6.0 with CUDA 12.4 in virtual environment"""
    global TORCH_VERSION, CUDA_VERSION, DETECTED_VRAM, PYTHON_PATH, PIP_PATH

    torch_version, cuda_version, index_url, vram_gb = detect_gpu_generation()
    print(f"\n🔥 Installing PyTorch {torch_version} with {cuda_version}...")
    print(f"📍 Using Python: {PYTHON_PATH}")
    print(f"📍 Using Pip: {PIP_PATH}")

    try:
        # Clean any existing PyTorch installations first
        print("🧹 Cleaning existing PyTorch installations...")
        uninstall_cmd = [PIP_PATH, 'uninstall', '-y', 'torch', 'torchvision', 'torchaudio']
        subprocess.run(uninstall_cmd, capture_output=True)

        # Install PyTorch 2.6.0 with CUDA 12.4
        install_cmd = [
            PIP_PATH, 'install',
            f'torch=={torch_version}', 'torchvision', 'torchaudio',
            '--index-url', index_url,
            '--timeout=600',
            '--no-cache-dir'  # Prevent cache conflicts
        ]

        if not USES_VIRTUAL_ENV:
            install_cmd.append('--user')

        print(f"🔧 Running: {' '.join(install_cmd)}")
        result = subprocess.run(install_cmd, capture_output=True, text=True, timeout=900)

        if result.returncode != 0:
            print("❌ PyTorch installation failed!")
            print(f"Error: {result.stderr}")
            return "unknown", "unknown", 8.0

        print("\n🔬 Verifying PyTorch installation...")
        verify_script = '''
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    if torch.cuda.device_count() > 0:
        print(f"GPU name: {torch.cuda.get_device_name(0)}")
        # Test basic CUDA operation
        x = torch.randn(2, 3).cuda()
        print(f"CUDA tensor test: {x.device}")
else:
    print("CUDA not available - check installation")
'''

        verify_result = subprocess.run([PYTHON_PATH, '-c', verify_script],
                                     capture_output=True, text=True, timeout=60)

        if verify_result.returncode == 0:
            print("✅ PyTorch installation and verification successful!")
            print(verify_result.stdout)
        else:
            print("⚠️ PyTorch installed but verification had issues:")
            print(verify_result.stderr)
            # Try alternative verification
            simple_verify = subprocess.run([PYTHON_PATH, '-c', 'import torch; print("Import successful")'],
                                         capture_output=True, text=True)
            if simple_verify.returncode == 0:
                print("✅ Basic PyTorch import works")
            else:
                print("❌ PyTorch import failed completely")

        TORCH_VERSION, CUDA_VERSION, DETECTED_VRAM = torch_version, cuda_version, vram_gb
        return torch_version, cuda_version, vram_gb

    except Exception as e:
        print(f"❌ PyTorch installation error: {e}")
        TORCH_VERSION, CUDA_VERSION, DETECTED_VRAM = "unknown", "unknown", 8.0
        return "unknown", "unknown", 8.0

# Execute
try:
    TORCH_VERSION, CUDA_VERSION, DETECTED_VRAM = install_pytorch()
    print(f"\n📊 Installation Summary:")
    print(f"  Platform: {DETECTED_PLATFORM}")
    print(f"  Environment: {'Virtual Env (Python 3.10.9)' if USES_VIRTUAL_ENV else 'System + --user'}")
    print(f"  PyTorch: {TORCH_VERSION}")
    print(f"  CUDA: {CUDA_VERSION}")
    print(f"  VRAM: {DETECTED_VRAM:.1f} GB")
except Exception as e:
    print(f"❌ Installation process failed: {e}")



🎮 Detecting GPU generation...
⚠️ GPU detection failed: 'list' object has no attribute 'strip'
📦 Defaulting to PyTorch 2.6.0 + CUDA 12.4

🔥 Installing PyTorch 2.6.0 with cu124...
📍 Using Python: /content/wan2gp_env/bin/python
📍 Using Pip: /content/wan2gp_env/bin/pip
🧹 Cleaning existing PyTorch installations...
🔧 Running: /content/wan2gp_env/bin/pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/test/cu124 --timeout=600 --no-cache-dir

🔬 Verifying PyTorch installation...
✅ PyTorch installation and verification successful!
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU count: 1
GPU name: Tesla T4
CUDA tensor test: cuda:0


📊 Installation Summary:
  Platform: colab
  Environment: Virtual Env (Python 3.10.9)
  PyTorch: 2.6.0
  CUDA: cu124
  VRAM: 8.0 GB


In [3]:
#@title 📦 Step 3: Clone Wan2GP Repository & Install Core Dependencies
import os
import subprocess

def clone_repository():
    """Clone repository with error handling"""
    print("📥 Cloning Wan2GP repository...")

    try:
        if os.path.exists('Wan2GP'):
            print("📁 Wan2GP directory already exists - updating...")
            os.chdir('Wan2GP')
            subprocess.run(['git', 'pull'], check=True)
            os.chdir('..')
        else:
            print("Cloning new repository...")
            subprocess.run(['git', 'clone', 'https://github.com/deepbeepmeep/Wan2GP.git'], check=True)

        if not os.path.exists('Wan2GP') or not os.listdir('Wan2GP'):
            raise Exception("Git clone failed to create the directory.")

        print("✅ Repository cloned/updated successfully")
        if not os.getcwd().endswith('Wan2GP'):
            os.chdir('Wan2GP')
        return True

    except Exception as e:
        print(f"❌ Primary clone method failed: {e}")
        print("🔄 Attempting alternative download...")
        try:
            print("Cleaning up old files...")
            subprocess.run(['rm', '-f', 'main.zip'], check=False)
            subprocess.run(['rm', '-rf', 'Wan2GP-main'], check=False)

            print("Downloading zip archive...")
            subprocess.run(['wget', '-q', 'https://github.com/deepbeepmeep/Wan2GP/archive/main.zip'], check=True)

            print("Unzipping archive...")
            subprocess.run(['unzip', '-o', '-q', 'main.zip'], check=True)

            print("Setting up directory...")
            if os.path.exists('Wan2GP'):
                subprocess.run(['rm', '-rf', 'Wan2GP'])
            subprocess.run(['mv', 'Wan2GP-main', 'Wan2GP'], check=True)

            os.chdir('Wan2GP')
            print("✅ Repository downloaded via wget")
            return True
        except Exception as fallback_e:
            print(f"❌ All download methods failed: {fallback_e}")
            return False

def install_dependencies():
    """Install dependencies in virtual environment"""
    global PIP_PATH, PYTHON_PATH
    print("\n📦 Installing core dependencies...")

    try:
        # Install from requirements.txt
        install_cmd = [PIP_PATH, 'install', '-r', 'requirements.txt', '--timeout=600']
        if not USES_VIRTUAL_ENV:
            install_cmd.append('--user')

        subprocess.run(install_cmd, check=True)
        print("✅ Core dependencies installation complete.")
        return True

    except Exception as e:
        print(f"⚠️ Main installation failed: {e}")
        print("Attempting to install critical dependencies manually...")

        critical_deps = [
            'diffusers>=0.31.0',
            'transformers==4.44.2',  # Stable version for Python 3.10.9
            'accelerate>=1.1.1',
            'gradio==5.23.0',
            'opencv-python>=4.9.0.80',
            'imageio',
            'einops',
            'safetensors',
            'tqdm'
        ]

        for dep in critical_deps:
            print(f"Installing fallback: {dep}...")
            fallback_cmd = [PIP_PATH, 'install', dep]
            if not USES_VIRTUAL_ENV:
                fallback_cmd.append('--user')
            try:
                subprocess.run(fallback_cmd, check=True)
            except:
                print(f"Failed to install {dep}")

        print("✅ Fallback installation complete.")
        return True

# Execute
try:
    repo_success = clone_repository()
    if repo_success:
        deps_success = install_dependencies()
        if deps_success:
            print(f"\n📍 Current directory: {os.getcwd()}")
            print(f"📁 Wan2GP files present: {os.path.exists('wgp.py')}")
        else:
            print("⚠️ Dependency installation failed.")
    else:
        print("⚠️ Repository cloning failed.")
except Exception as e:
    print(f"❌ Setup failed: {e}")


📥 Cloning Wan2GP repository...
📁 Wan2GP directory already exists - updating...
✅ Repository cloned/updated successfully

📦 Installing core dependencies...
✅ Core dependencies installation complete.

📍 Current directory: /content/Wan2GP
📁 Wan2GP files present: True


In [4]:
#@title ⚡ Step 4: Performance Optimizations - Production Ready (Python 3.10.9)
import subprocess
import os

def safe_install_production(package_command, description, test_import=None, fallback_packages=None):
    """Production-grade installation with comprehensive fallbacks"""
    global PIP_PATH, PYTHON_PATH
    print(f"\n{'='*60}")
    print(f"🔧 Installing: {description}")
    print(f"{'='*60}")

    # Prepare install command
    if isinstance(package_command, list):
        command_to_run = [PIP_PATH] + package_command
    else:
        command_to_run = [PIP_PATH, 'install'] + package_command.split()

    if not USES_VIRTUAL_ENV:
        command_to_run.append('--user')

    print(f" ▶️ Primary attempt: {' '.join(command_to_run)}")

    # Try primary installation
    try:
        result = subprocess.run(command_to_run, capture_output=True, text=True, timeout=600)

        if result.returncode == 0:
            print(f" ✅ Primary installation successful: {description}")

            # Verify import if specified
            if test_import:
                print(f" ▶️ Verifying import: {test_import}")
                verify_result = subprocess.run(
                    [PYTHON_PATH, '-c', f'import {test_import}; print("✅ {test_import} verified")'],
                    capture_output=True, text=True, timeout=30
                )
                if verify_result.returncode == 0:
                    print(f" ✅ Import verification successful: {description}")
                    return True
                else:
                    print(f" ⚠️ Import failed but package installed: {description}")
            else:
                return True
        else:
            print(f" ❌ Primary installation failed: {description}")
            print(f" Error preview: {result.stderr[:200]}...")

    except subprocess.TimeoutExpired:
        print(f" ⏰ Primary installation timed out: {description}")
    except Exception as e:
        print(f" ❌ Primary installation exception: {e}")

    # Try fallback packages if provided
    if fallback_packages:
        print(f" 🔄 Attempting fallback installations...")
        for i, fallback_pkg in enumerate(fallback_packages, 1):
            try:
                print(f" ▶️ Fallback {i}: {fallback_pkg}")
                fallback_cmd = [PIP_PATH, 'install'] + fallback_pkg.split()
                if not USES_VIRTUAL_ENV:
                    fallback_cmd.append('--user')

                fallback_result = subprocess.run(fallback_cmd, capture_output=True, text=True, timeout=300)

                if fallback_result.returncode == 0:
                    print(f" ✅ Fallback {i} successful: {description}")

                    # Quick import test for fallback
                    if test_import:
                        verify_result = subprocess.run(
                            [PYTHON_PATH, '-c', f'import {test_import}; print("Fallback import OK")'],
                            capture_output=True, text=True, timeout=10
                        )
                        if verify_result.returncode == 0:
                            return True
                    else:
                        return True

            except Exception as fallback_e:
                print(f" ❌ Fallback {i} failed: {fallback_e}")
                continue

    print(f" ❌ All installation attempts failed for: {description}")
    return False

def install_performance_optimizations_production():
    """Production-ready performance optimization installation"""
    global BEST_ATTENTION, optimization_results

    print("🚀 Installing Production-Ready Performance Optimizations...")
    print("🎯 Focus: Maximum compatibility with WanGP + Python 3.10.9 ecosystem")

    optimization_results = {
        'xformers': False,
        'sage_v1': False,
        'triton': False,
        'flash': False
    }

    # Clean existing installations first
    print("\n🧹 Cleaning conflicting installations...")
    cleanup_packages = ['sageattention', 'xformers', 'flash-attn', 'triton']
    for pkg in cleanup_packages:
        try:
            subprocess.run([PIP_PATH, 'uninstall', pkg, '-y'],
                          capture_output=True, text=True, timeout=30)
        except:
            pass

    # Clear pip cache
    try:
        subprocess.run([PIP_PATH, 'cache', 'purge'], capture_output=True, text=True)
        print("✅ Environment cleaned")
    except:
        print("⚠️ Cache clear failed, continuing...")

    # 1. Install Triton (essential for SageAttention)
    print("\n📦 Installing Triton (Foundation for SageAttention)...")
    triton_success = safe_install_production(
        'triton',
        'Triton Compiler',
        test_import='triton',
        fallback_packages=['triton --pre', 'triton --no-deps']
    )
    optimization_results['triton'] = triton_success

    # 2. Install xFormers (highest compatibility, universal support)
    print("\n📦 Installing xFormers (Universal GPU Compatibility)...")
    xformers_success = safe_install_production(
        'xformers --no-deps',
        'xFormers Memory Efficient Attention',
        test_import='xformers',
        fallback_packages=['xformers', 'xformers --pre --no-deps']
    )
    optimization_results['xformers'] = xformers_success

    # 3. Install SageAttention v1.0.6 (stable, 30% boost, Python 3.10.9 compatible)
    print("\n📦 Installing SageAttention v1.0.6 (30% Boost, Production Stable)...")
    sage_v1_success = safe_install_production(
        'sageattention==1.0.6 --no-cache-dir',
        'SageAttention v1.0.6 (Python 3.10.9 Compatible)',
        test_import='sageattention',
        fallback_packages=[
            'sageattention==1.0.6 --no-deps',
            'sageattention==1.0.5 --no-cache-dir',
            'sageattention --no-build-isolation'
        ]
    )
    optimization_results['sage_v1'] = sage_v1_success

    # 4. Optional: Flash Attention 2 (often problematic on Colab, but try anyway)
    print("\n📦 Attempting Flash Attention 2 (Optional - Often Fails on Colab)...")
    try:
        flash_success = safe_install_production(
            'flash-attn==2.6.3 --no-build-isolation',
            'Flash Attention 2 (Optional)',
            test_import='flash_attn',
            fallback_packages=[
                'flash-attn --no-build-isolation --no-cache-dir',
                'flash-attn==2.5.9 --no-build-isolation'
            ]
        )
        optimization_results['flash'] = flash_success
    except:
        print("⚠️ Flash Attention installation skipped due to compilation complexity")
        optimization_results['flash'] = False

    # Results Analysis and Best Attention Selection
    print("\n" + "="*70)
    print("🏆 PRODUCTION PERFORMANCE OPTIMIZATION RESULTS")
    print("="*70)

    results_summary = {
        'triton': ('Triton Compiler', 'Essential for SageAttention', optimization_results['triton']),
        'xformers': ('xFormers', 'Universal GPU support, stable performance', optimization_results['xformers']),
        'sage_v1': ('SageAttention v1.0.6', 'Python 3.10.9 compatible, 30% boost', optimization_results['sage_v1']),
        'flash': ('Flash Attention 2', 'High performance (complex installation)', optimization_results['flash'])
    }

    installed_count = 0
    for key, (name, description, status) in results_summary.items():
        status_emoji = "✅ INSTALLED" if status else "❌ FAILED"
        print(f" {status_emoji}: {name}")
        print(f"     └─ {description}")
        if status:
            installed_count += 1

    # Intelligent Best Attention Selection
    if optimization_results['sage_v1']:
        BEST_ATTENTION = "sage"
        performance_desc = "SageAttention v1.0.6 (Stable Performance - 30% boost)"
        print(f"\n🎯 SELECTED: {performance_desc}")
        print("   ✨ Reliable and proven performance optimization")

    elif optimization_results['xformers']:
        BEST_ATTENTION = "xformers"
        performance_desc = "xFormers (Stable, Universal Compatibility)"
        print(f"\n🎯 SELECTED: {performance_desc}")
        print("   🛡️ Most compatible attention mechanism")

    elif optimization_results['flash']:
        BEST_ATTENTION = "flash"
        performance_desc = "Flash Attention 2 (High Performance)"
        print(f"\n🎯 SELECTED: {performance_desc}")
        print("   🔥 Advanced attention with good performance")

    else:
        BEST_ATTENTION = "sdpa"
        performance_desc = "SDPA (PyTorch Default - Fallback)"
        print(f"\n🎯 SELECTED: {performance_desc}")
        print("   🏠 Reliable fallback option")

    print(f"\n📊 Final Status: {installed_count}/{len(results_summary)} optimizations installed")
    print("="*70)

    # Store results for launch configuration
    os.environ['WAN2GP_BEST_ATTENTION'] = BEST_ATTENTION
    os.environ['WAN2GP_OPTIMIZATION_COUNT'] = str(installed_count)

    return installed_count > 0

# Execute the production installation
print("🎬 Starting Production Performance Optimization Installation")
print("🎯 Optimized for WanGP + Python 3.10.9 + Google Colab")
print("✅ Using SageAttention v1.0.6 for maximum stability")

success = install_performance_optimizations_production()

if success:
    print(f"\n✅ Production performance optimization setup complete!")
    print(f"🚀 Recommended attention mechanism: {BEST_ATTENTION}")
    print(f"📝 Use --attention {BEST_ATTENTION} when launching WanGP")

    # Quick launch command suggestion
    print(f"\n💡 Suggested launch command:")
    print(f"   {PYTHON_PATH} wgp.py --attention {BEST_ATTENTION} --profile 4 --compile")
else:
    print(f"\n⚠️ Some optimizations failed, but WanGP will work with basic SDPA attention")
    print(f"🔄 Fallback launch command:")
    print(f"   {PYTHON_PATH} wgp.py --attention sdpa --profile 4")

print(f"\n🎉 Setup complete! Your WanGP installation is ready for optimal performance.")


🎬 Starting Production Performance Optimization Installation
🎯 Optimized for WanGP + Python 3.10.9 + Google Colab
✅ Using SageAttention v1.0.6 for maximum stability
🚀 Installing Production-Ready Performance Optimizations...
🎯 Focus: Maximum compatibility with WanGP + Python 3.10.9 ecosystem

🧹 Cleaning conflicting installations...
✅ Environment cleaned

📦 Installing Triton (Foundation for SageAttention)...

🔧 Installing: Triton Compiler
 ▶️ Primary attempt: /content/wan2gp_env/bin/pip install triton
 ✅ Primary installation successful: Triton Compiler
 ▶️ Verifying import: triton
 ✅ Import verification successful: Triton Compiler

📦 Installing xFormers (Universal GPU Compatibility)...

🔧 Installing: xFormers Memory Efficient Attention
 ▶️ Primary attempt: /content/wan2gp_env/bin/pip install xformers --no-deps
 ✅ Primary installation successful: xFormers Memory Efficient Attention
 ▶️ Verifying import: xformers
 ✅ Import verification successful: xFormers Memory Efficient Attention

📦 Inst

In [5]:
#@title 🎨 Step 5: LoRA Management System
import os
import glob
import subprocess

class ProductionLoRAManager:
    def __init__(self):
        self.lora_directories = {
            'Text2Video': 'loras/',
            'Image2Video': 'loras_i2v/',
            'Hunyuan_T2V': 'loras_hunyuan/',
            'Hunyuan_I2V': 'loras_hunyuan_i2v/',
            'LTX_Video': 'loras_ltxv/',
        }
        self.setup_directories()
        self.download_essential_loras()

    def setup_directories(self):
        print("\n🔧 Setting up LoRA directories...")
        for name, path in self.lora_directories.items():
            os.makedirs(path, exist_ok=True)
            print(f"  ✅ Directory ensured: ./{path}")

    def download_essential_loras(self):
        print("\n📥 Downloading essential accelerator LoRAs...")
        essential_loras = {
            'loras/': [
                'https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan21_AccVid_T2V_14B_lora_rank32_fp16.safetensors'
            ],
            'loras_i2v/': [
                'https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan21_AccVid_I2V_480P_14B_lora_rank32_fp16.safetensors'
            ]
        }

        for directory, urls in essential_loras.items():
            for url in urls:
                filepath = os.path.join(directory, os.path.basename(url))
                if not os.path.exists(filepath):
                    print(f"  ▶️ Downloading {os.path.basename(url)}...")
                    try:
                        subprocess.run(['wget', '-q', url, '-O', filepath], check=True)
                        print(f"  ✅ Download complete.")
                    except:
                        print(f"  ❌ Download FAILED.")
                else:
                    print(f"  ✅ {os.path.basename(url)} already exists, skipping.")

    def get_lora_args(self):
        print("\n [LoRA Debug] Scanning for LoRA directories...")
        args = []
        arg_map = {
            'Text2Video': '--lora-dir',
            'Image2Video': '--lora-dir-i2v',
            'Hunyuan_T2V': '--lora-dir-hunyuan',
            'Hunyuan_I2V': '--lora-dir-hunyuan-i2v',
            'LTX_Video': '--lora-dir-ltxv'
        }

        for key, path in self.lora_directories.items():
            if os.path.exists(path) and any(f.endswith(('.safetensors', '.pt', '.pth'))
                                          for f in os.listdir(path)):
                arg_name = arg_map[key]
                args.extend([arg_name, path])
                print(f"  Found LoRAs in '{path}', adding: {arg_name}")
        return args

    def get_lora_summary(self):
        """Get summary of available LoRAs"""
        print("\n📊 LoRA Inventory Summary:")
        total_loras = 0

        for name, path in self.lora_directories.items():
            if os.path.exists(path):
                lora_files = [f for f in os.listdir(path)
                             if f.endswith(('.safetensors', '.pt', '.pth'))]
                count = len(lora_files)
                total_loras += count

                if count > 0:
                    print(f"  📁 {name}: {count} LoRA(s)")
                    for lora in lora_files[:3]:  # Show first 3
                        print(f"    └─ {lora}")
                    if count > 3:
                        print(f"    └─ ... and {count - 3} more")
                else:
                    print(f"  📁 {name}: 0 LoRAs")

        print(f"\n🎯 Total LoRAs available: {total_loras}")
        return total_loras

# Create LoRA manager
lora_manager = ProductionLoRAManager()
lora_count = lora_manager.get_lora_summary()
print(f"\n✅ LoRA management system ready with {lora_count} LoRAs.")



🔧 Setting up LoRA directories...
  ✅ Directory ensured: ./loras/
  ✅ Directory ensured: ./loras_i2v/
  ✅ Directory ensured: ./loras_hunyuan/
  ✅ Directory ensured: ./loras_hunyuan_i2v/
  ✅ Directory ensured: ./loras_ltxv/

📥 Downloading essential accelerator LoRAs...
  ✅ Wan21_AccVid_T2V_14B_lora_rank32_fp16.safetensors already exists, skipping.
  ✅ Wan21_AccVid_I2V_480P_14B_lora_rank32_fp16.safetensors already exists, skipping.

📊 LoRA Inventory Summary:
  📁 Text2Video: 1 LoRA(s)
    └─ Wan21_AccVid_T2V_14B_lora_rank32_fp16.safetensors
  📁 Image2Video: 1 LoRA(s)
    └─ Wan21_AccVid_I2V_480P_14B_lora_rank32_fp16.safetensors
  📁 Hunyuan_T2V: 0 LoRAs
  📁 Hunyuan_I2V: 0 LoRAs
  📁 LTX_Video: 0 LoRAs

🎯 Total LoRAs available: 2

✅ LoRA management system ready with 2 LoRAs.


In [6]:
#@title 🚀 Step 6: Production Launcher with Optimized Settings
import sys
import os
import subprocess
import time

def fix_gradio_compatibility():
    """Fix common Gradio compatibility issues for production"""
    print("🔧 Applying Gradio compatibility fixes...")

    try:
        wgp_path = 'wgp.py'
        if os.path.exists(wgp_path):
            with open(wgp_path, 'r') as f:
                content = f.read()

            # Fix Row height parameter issues (common in Gradio updates)
            import re
            original_content = content

            # Multiple patterns to fix Row height issues
            patterns = [
                (r'gr\.Row$$height=\d+$$', 'gr.Row()'),
                (r'gr\.Row$$height=\d+,\s*([^)]+)$$', r'gr.Row(\1)'),
                (r'gr\.Row$$\s*height=\d+\s*,\s*([^)]+)$$', r'gr.Row(\1)'),
                (r'gr\.Row$$\s*([^),]+),\s*height=\d+\s*$$', r'gr.Row(\1)'),
            ]

            for pattern, replacement in patterns:
                content = re.sub(pattern, replacement, content)

            if content != original_content:
                # Create backup
                subprocess.run(['cp', wgp_path, f'{wgp_path}.backup'], check=False)

                with open(wgp_path, 'w') as f:
                    f.write(content)
                print("✅ Fixed Gradio Row compatibility issues")
            else:
                print("✅ No Gradio compatibility fixes needed")

    except Exception as e:
        print(f"⚠️ Gradio fix failed: {e}")

def generate_production_command():
    """Generate production-optimized launch command"""
    global DETECTED_VRAM, BEST_ATTENTION, PYTHON_PATH

    vram_gb = DETECTED_VRAM if 'DETECTED_VRAM' in globals() else 8.0
    attention = BEST_ATTENTION if 'BEST_ATTENTION' in globals() else 'sdpa'

    # Enhanced configuration based on hardware
    config = {'attention': attention}

    if vram_gb >= 16:
        config.update({
            'model': 't2v-14B',
            'profile': 3,
            'desc': "High VRAM (≥16GB) - Maximum Quality"
        })
    elif vram_gb >= 10:
        config.update({
            'model': 't2v-14B',
            'profile': 4,
            'desc': "Medium VRAM (10-15GB) - Balanced"
        })
    else:
        config.update({
            'model': 't2v-1-3B',
            'profile': 4,
            'desc': "Standard VRAM (<10GB) - Efficient"
        })

    # Enable compilation for advanced attention
    if attention in ['sage', 'xformers'] and 'triton' in [pkg for pkg in optimization_results if optimization_results[pkg]]:
        config['compile'] = True

    # Configuration summary
    print("\n" + "="*60)
    print("🎯 PRODUCTION LAUNCH CONFIGURATION")
    print(f"  🖥️ VRAM: {vram_gb:.1f} GB ({config['desc']})")
    print(f"  🧠 Model: {config['model']}")
    print(f"  ⚡ Attention: '{attention}'")
    print(f"  🔧 PyTorch Compile: {'✅ Enabled' if config.get('compile') else '❌ Disabled'}")
    print(f"  📊 Memory Profile: {config['profile']}")
    print(f"  🐍 Python: {PYTHON_PATH}")
    print("="*60)

    # Build command with production options
    cmd_parts = [
        PYTHON_PATH, "wgp.py",
        f"--{config['model']}",
        "--profile", str(config['profile']),
        "--attention", attention,
        "--advanced",
        "--server-port", "7860",
        "--share",
        "--listen",
        "--open-browser"
    ]

    # Add compilation if supported
    if config.get('compile'):
        cmd_parts.append("--compile")

    # Add LoRA directories if available
    lora_args = lora_manager.get_lora_args()
    if lora_args:
        cmd_parts.extend(lora_args)
        cmd_parts.append("--check-loras")

    # Add production stability options
    cmd_parts.extend([
        "--verbose", "1",  # Moderate verbosity
        "--fp16"  # Force FP16 for compatibility
    ])

    return " ".join(cmd_parts)

def launch_production():
    """Launch WanGP with production settings"""

    print("\n🚀 LAUNCHING WAN2GP - PRODUCTION READY")
    print("="*60)
    print("🎯 Configuration: Python 3.10.9 + SageAttention v1.0.6")
    print("✨ Expected performance: 30% boost with maximum stability")
    print("="*60)

    # Ensure correct directory
    if not os.path.exists('wgp.py') and os.path.exists('Wan2GP/wgp.py'):
        print("📁 Changing to Wan2GP directory...")
        os.chdir('Wan2GP')

    # Apply fixes and generate command
    fix_gradio_compatibility()
    launch_command = generate_production_command()

    print(f"\n📋 Launch Command:")
    print(f"{launch_command}")
    print("-" * 60)

    try:
        print("🎬 Starting WanGP...")
        print("📡 Monitoring for Gradio interface...")

        # Launch with real-time output
        process = subprocess.Popen(
            launch_command.split(),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True,
            bufsize=1
        )

        # Monitor initial output
        gradio_links = []
        start_time = time.time()

        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break

            if output:
                print(output.strip())

                # Capture Gradio links
                if "Running on" in output:
                    gradio_links.append(output.strip())
                    if "https://" in output:  # Public link found
                        print("✅ Public Gradio link detected!")
                        break

                # Check for successful startup
                if "Startup complete" in output or "Interface launched" in output:
                    print("✅ WanGP launched successfully!")
                    break

            # Timeout check
            if time.time() - start_time > 120:  # 2 minute timeout
                print("⏰ Startup monitoring timeout - WanGP may still be starting...")
                break

        # Display discovered links
        if gradio_links:
            print("\n🔗 Available Gradio Interfaces:")
            for link in gradio_links:
                print(f"   {link}")
            print("\n🎉 WanGP is ready! Use the links above to access the interface.")
        else:
            print("\n⚠️ No Gradio links detected, but process may be running.")
            print("   Check above output for any error messages.")

        return True

    except KeyboardInterrupt:
        print("\n⏹️ Launch interrupted by user")
        return False
    except Exception as e:
        print(f"\n❌ Launch failed: {e}")
        print(f"\n🔄 Manual launch command:")
        print(f"{launch_command}")
        return False

# Execute production launch
try:
    print("🎬 Initializing Production WanGP Launch...")

    # Quick environment check
    if 'PYTHON_PATH' not in globals():
        print("⚠️ Environment variables not set - using fallback")
        PYTHON_PATH = '/content/wan2gp_env/bin/python'
        DETECTED_VRAM = 8.0
        BEST_ATTENTION = 'sdpa'

    success = launch_production()

    if success:
        print("\n🎊 WanGP Production Launch Complete!")
        print("💡 Tips:")
        print("   -  Start with smaller models (1.3B) to test your setup")
        print("   -  Use 14B models for final high-quality output")
        print("   -  Check the LoRA section for style customizations")
    else:
        print("\n🔧 If launch failed, try the emergency fallback:")
        print(f"   {PYTHON_PATH} wgp.py --attention sdpa --profile 4 --share")

except Exception as e:
    print(f"❌ Production launch setup failed: {e}")
    print("\n🆘 Emergency launch command:")
    print("   python wgp.py --t2v-1-3B --attention sdpa --profile 4 --share")


🎬 Initializing Production WanGP Launch...

🚀 LAUNCHING WAN2GP - PRODUCTION READY
🎯 Configuration: Python 3.10.9 + SageAttention v1.0.6
✨ Expected performance: 30% boost with maximum stability
🔧 Applying Gradio compatibility fixes...
✅ No Gradio compatibility fixes needed

🎯 PRODUCTION LAUNCH CONFIGURATION
  🖥️ VRAM: 8.0 GB (Standard VRAM (<10GB) - Efficient)
  🧠 Model: t2v-1-3B
  ⚡ Attention: 'sage'
  🔧 PyTorch Compile: ✅ Enabled
  📊 Memory Profile: 4
  🐍 Python: /content/wan2gp_env/bin/python

 [LoRA Debug] Scanning for LoRA directories...
  Found LoRAs in 'loras/', adding: --lora-dir
  Found LoRAs in 'loras_i2v/', adding: --lora-dir-i2v

📋 Launch Command:
/content/wan2gp_env/bin/python wgp.py --t2v-1-3B --profile 4 --attention sage --advanced --server-port 7860 --share --listen --open-browser --compile --lora-dir loras/ --lora-dir-i2v loras_i2v/ --check-loras --verbose 1 --fp16
------------------------------------------------------------
🎬 Starting WanGP...
📡 Monitoring for Gradio in

In [7]:
#@title 🔄 Alternative Package Version Fix
import subprocess

def downgrade_transformers():
    """Use older transformers version for better compatibility"""
    print("🔄 Installing compatible package versions...")

    packages_to_fix = [
        'transformers==4.44.2',  # Older, more stable version
        'torchvision==0.18.1',   # Compatible version
        'optimum==1.21.4'        # Stable optimum version
    ]

    for package in packages_to_fix:
        print(f"📦 Installing {package}...")
        result = subprocess.run([
            '/content/wan2gp_py312_env/bin/pip', 'install',
            package, '--force-reinstall', '--no-cache-dir'
        ], capture_output=True, text=True)

        if result.returncode == 0:
            print(f"✅ {package} installed successfully")
        else:
            print(f"⚠️ {package} installation had issues")

downgrade_transformers()


🔄 Installing compatible package versions...
📦 Installing transformers==4.44.2...


FileNotFoundError: [Errno 2] No such file or directory: '/content/wan2gp_py312_env/bin/pip'

In [ ]:
#@title 🏃‍♂️ Quick Launch with Environment Fix
import os
import subprocess

# Set environment variables to avoid torchvision issues
os.environ['PYTORCH_DISABLE_TORCHVISION_FUNCTOR'] = '1'
os.environ['TORCHVISION_DISABLE_META_REGISTRATIONS'] = '1'

# Simple launch command without advanced features
launch_cmd = [
    '/content/wan2gp_py312_env/bin/python', 'wgp.py',
    '--t2v-1-3B',           # Use smaller model to reduce complexity
    '--attention', 'sdpa',   # Use stable attention
    '--profile', '4',        # Conservative memory profile
    '--server-port', '7863', # Different port
    '--share',              # Enable sharing
    '--verbose', '1'        # Reduce verbosity
]

print("🚀 Launching with compatibility workarounds...")
print(f"📋 Command: {' '.join(launch_cmd)}")

# Change to correct directory
if not os.getcwd().endswith('Wan2GP'):
    os.chdir('/content/Wan2GP')

# Launch with environment fixes
subprocess.run(launch_cmd)


In [ ]:
#@title 🚀 Step 6: Enhanced Launcher with Gradio Fixes & Robust Link Generation
import sys
import os
import subprocess
import time

def fix_gradio_compatibility():
    """Fix common Gradio compatibility issues"""
    print("🔧 Fixing Gradio compatibility issues...")

    try:
        # Fix the common gr.Row(height=X) compatibility issue
        wgp_path = 'wgp.py'
        if os.path.exists(wgp_path):
            with open(wgp_path, 'r') as f:
                content = f.read()

            # Fix Row height parameter issues
            import re
            original_content = content

            # Multiple patterns to fix Row height issues
            patterns = [
                (r'gr\.Row\(height=\d+\)', 'gr.Row()'),
                (r'gr\.Row\(height=\d+,\s*([^)]+)\)', r'gr.Row(\1)'),
                (r'gr\.Row\(\s*height=\d+\s*,\s*([^)]+)\)', r'gr.Row(\1)'),
                (r'gr\.Row\(\s*([^),]+),\s*height=\d+\s*\)', r'gr.Row(\1)'),
            ]

            for pattern, replacement in patterns:
                content = re.sub(pattern, replacement, content)

            if content != original_content:
                # Create backup
                subprocess.run(['cp', wgp_path, f'{wgp_path}.backup'], check=False)

                with open(wgp_path, 'w') as f:
                    f.write(content)
                print("✅ Fixed Gradio Row compatibility issues")
            else:
                print("✅ No Gradio compatibility fixes needed")

    except Exception as e:
        print(f"⚠️ Gradio fix failed: {e}")

def generate_optimized_command():
    """Generate optimized launch command with robust settings"""
    global DETECTED_VRAM, BEST_ATTENTION, PYTHON_PATH

    vram_gb = DETECTED_VRAM
    attention = BEST_ATTENTION

    # Enhanced configuration based on hardware
    config = {'attention': attention}

    if vram_gb >= 16:
        config.update({
            'model': 't2v-14B',
            'profile': 3,
            'desc': "High VRAM (≥16GB) - Maximum Quality"
        })
    elif vram_gb >= 10:
        config.update({
            'model': 't2v-14B',
            'profile': 4,
            'desc': "Medium VRAM (10-15GB) - Balanced"
        })
    else:
        config.update({
            'model': 't2v-1-3B',
            'profile': 4,
            'desc': "Standard VRAM (<10GB) - Efficient"
        })

    # Enable compilation for advanced attention
    if attention in ['sage', 'sage2', 'xformers']:
        config['compile'] = True

    # Configuration summary
    print("\n" + "="*60)
    print("🎯 OPTIMIZED LAUNCH CONFIGURATION")
    print(f"  🖥️  VRAM: {vram_gb:.1f} GB ({config['desc']})")
    print(f"  🧠 Model: {config['model']}")
    print(f"  ⚡ Attention: '{attention}'")
    print(f"  🔧 PyTorch Compile: {'✅ Enabled' if config.get('compile') else '❌ Disabled'}")
    print(f"  📊 Memory Profile: {config['profile']}")
    print(f"  🐍 Python: {PYTHON_PATH}")
    print("="*60)

    # Build command with enhanced options
    cmd_parts = [
        PYTHON_PATH, "wgp.py",
        f"--{config['model']}",
        "--profile", str(config['profile']),
        "--attention", attention,
        "--advanced",
        "--server-port", "7861",  # Use alternative port to avoid conflicts
        "--share",
        "--listen",  # Enable network access
        "--open-browser"  # Auto-open browser
    ]

    # Add compilation if supported
    if config.get('compile'):
        cmd_parts.append("--compile")

    # Add LoRA directories if available
    lora_args = lora_manager.get_lora_args()
    if lora_args:
        cmd_parts.extend(lora_args)
        cmd_parts.append("--check-loras")

    # Add additional stability options
    cmd_parts.extend([
        "--verbose", "2",  # Verbose output for debugging
        "--fp16"  # Force FP16 for better compatibility
    ])

    return " ".join(cmd_parts)

def launch_with_retry(command, max_attempts=3):
    """Launch WanGP with retry mechanism and better error handling"""
    for attempt in range(max_attempts):
        print(f"\n🚀 Launch Attempt {attempt + 1}/{max_attempts}")
        print(f"📋 Command: {command}")
        print("-" * 60)

        try:
            # Use Popen for better control and output capture
            process = subprocess.Popen(
                command.split(),
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1
            )

            # Monitor initial output for errors
            start_time = time.time()
            gradio_link_found = False

            print("📡 WanGP Starting... Monitoring for Gradio link...")

            while True:
                output = process.stdout.readline()
                if output == '' and process.poll() is not None:
                    break

                if output:
                    print(output.strip())

                    # Check for Gradio link
                    if "Running on" in output and "gradio" in output.lower():
                        gradio_link_found = True
                        print("✅ Gradio link detected!")
                        break

                    # Check for common errors
                    if any(error in output.lower() for error in ["error", "failed", "exception"]):
                        if "alsa" not in output.lower():  # Ignore ALSA warnings
                            print(f"⚠️ Potential error detected: {output.strip()}")

                # Timeout check (60 seconds to start)
                if time.time() - start_time > 60:
                    print("⏰ Startup timeout reached")
                    break

            if gradio_link_found:
                print("✅ WanGP launched successfully with Gradio interface!")
                return True
            else:
                print("⚠️ Gradio link not detected, but process may be running")
                if attempt < max_attempts - 1:
                    print("🔄 Retrying with different settings...")
                    process.terminate()
                    time.sleep(2)
                else:
                    return False

        except Exception as e:
            print(f"❌ Launch attempt {attempt + 1} failed: {e}")
            if attempt < max_attempts - 1:
                time.sleep(3)
            else:
                return False

    return False

# Ensure correct directory
if not os.path.exists('wgp.py') and os.path.exists('Wan2GP/wgp.py'):
    print("📁 Changing to Wan2GP directory...")
    os.chdir('Wan2GP')

# Apply fixes and launch
try:
    # Fix Gradio compatibility
    fix_gradio_compatibility()

    # Generate optimized command
    launch_command = generate_optimized_command()

    print("\n" + "="*60)
    print("🚀 LAUNCHING WAN2GP WITH ENHANCED SETTINGS")
    print("="*60)

    # Launch with retry mechanism
    success = launch_with_retry(launch_command)

    if not success:
        print("\n🔄 Primary launch failed. Trying emergency fallback...")

        # Emergency fallback command
        fallback_cmd = f"{PYTHON_PATH} wgp.py --t2v-1-3B --attention sdpa --profile 4 --server-port 7862 --share --listen --verbose 2"
        print(f"🆘 Emergency command: {fallback_cmd}")

        if launch_with_retry(fallback_cmd, max_attempts=1):
            print("✅ Emergency fallback successful!")
        else:
            print("❌ All launch attempts failed")
            print("\n🔍 Troubleshooting suggestions:")
            print("1. Check if port 7861/7862 is already in use")
            print("2. Try manually: python wgp.py --help")
            print("3. Check firewall/network settings")
            print("4. Restart Colab runtime if needed")

except Exception as e:
    print(f"❌ Launch process failed: {e}")
    print("\n🔄 Manual launch command:")
    print(f"{PYTHON_PATH} wgp.py --attention sdpa --profile 4 --server-port 7863 --share")


In [ ]:
#@title 🔑 HuggingFace Token Management (Fixed)
import os
import json
from pathlib import Path
import subprocess

# Fix the invalid token that was hardcoded
print("🔑 HuggingFace Token Manager")
print("=" * 40)

# Your current token appears to be invalid, so let's set it up properly
def setup_hf_token():
    """Setup HuggingFace token properly"""
    print("To use Wan2GP, you need a HuggingFace token.")
    print("\n📝 Steps to get your token:")
    print("1. Go to https://huggingface.co/settings/tokens")
    print("2. Create a new token with 'Read' permissions")
    print("3. Copy the token and paste below")

    # For Colab, users can enter their token
    from getpass import getpass
    try:
        token = getpass("🔐 Enter your HuggingFace token (hidden input): ")
        if token and token.startswith('hf_'):
            # Set environment variables
            os.environ['HF_TOKEN'] = token
            os.environ['HUGGINGFACE_HUB_TOKEN'] = token

            # Verify token
            import requests
            headers = {'Authorization': f'Bearer {token}'}
            response = requests.get('https://huggingface.co/api/whoami', headers=headers, timeout=10)

            if response.status_code == 200:
                user_info = response.json()
                print(f"✅ Token valid for user: {user_info.get('name', 'Unknown')}")
                return True
            else:
                print("❌ Token verification failed")
                return False
        else:
            print("❌ Invalid token format. Token should start with 'hf_'")
            return False
    except Exception as e:
        print(f"❌ Token setup failed: {e}")
        return False

# Remove the invalid hardcoded token and set up properly
if not setup_hf_token():
    print("\n⚠️  Continuing without HuggingFace token")
    print("   Some models may not be accessible")
